# Forecasting

[**This**](https://storage.googleapis.com/kagglesdsdata/competitions/86788/9875533/AirtrafficA4.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1730022090&Signature=eL0VEAJ0rKGKmirrmOOSG0wurd3VRxHQmXAbVSRMUMjf%2BOydILH37mrZYLOuYMV2RDOp0TJofFGR5%2BUuDSC4pFd%2Bc3ravjhozkusq8akfzk7j1uIW%2BSSYxN5EQr3y8yREJnPUbvvMq0iHXoYmKXGFWbIGGipcTWJk1MdUpLSIVRMcDpkmzWjzpcjPAtYlir77IxTWokjK4jy0%2FyUaqNpMCJ9jfR28o3AZsOKnmgrPZYvPPgVLi65lrOR8RMZgwRiZDYYCbSOjDOZpyiVysYWFElrzzRRhFbVBdY3BHv%2FnpQ%2FuyiTezz5OPGVRCkvJS%2FEHTeFPrHyGgK0lGGG4vmG0g%3D%3D&response-content-disposition=attachment%3B+filename%3DAirtrafficA4.csv) dataset provides **monthly operational metrics for a major Indian airline from 2013**.

It includes information on the **number of departures, flight hours, distance flown, passenger traffic, available seat kilometers, freight carried, and mail carried**. You can use any popular **time series library** for this task. (`Sktime`, `u8darts`, `GluonTS`, etc.)

Do the necessary **Feature Engineering/Data Cleanup** required to create time-series in your choice of a library.

In [1]:
import pandas as pd

# Load the dataset
airline_data = pd.read_csv('AirtrafficA4.csv')
airline_data.head(5)

,AIRLINE,YEAR,MONTH,TOTAL DEPARTURES,HOURS FLOWN,KILOMETRE FLOWN,PASSENGERS CARRIED,AVAILABLE SEAT KILOMETRE (IN THOUSAND),FREIGHT CARRIED (IN TONNE),MAIL CARRIED (IN TONNE)
0,A007,2023,JAN,"47,977","83,764","41,827","68,47,384","78,32,254","16,881.70","2,043.5"
1,A007,2023,FEB,"44,905","77,936","39,121","67,41,948","73,36,614","17,439.30","2,086.7"
2,A007,2023,MAR,"50,389","87,296","43,793","73,17,288","82,15,681","20,208.40","2,310.1"
3,A007,2023,APR,"48,752","84,232","42,615","74,06,440","80,05,648","19,432.80","2,102.9"
4,A007,2023,MAY,"50,956","87,917","44,505","81,09,626","83,75,201","24,165.10","2,102.4"
...,...,...,...,...,...,...,...,...,...,...
123,A007,2013,AUG,"12,278","21,571","11,884","15,31,406","21,39,007","9,600.00",0
124,A007,2013,SEP,"12,185","21,281","11,749","13,78,691","21,14,792","8,492.00",0
125,A007,2013,OCT,"12,780","22,345","12,296","15,10,184","22,13,139","9,111.00",0
126,A007,2013,NOV,"12,357","21,837","11,904","14,67,763","21,42,765","7,185.00",0


## Part 1
You need to predict ‘PASSENGERS CARRIED’ from 2023 Sep to 2024 Aug.

- **Part A:** Only Models discussed in class or its variant are allowed for submission for the competition leaderboard. Try to beat the benchmark submission. (Naive Drift)
- **Part B:** Use an LLM to generate predictions see section 3 of LLMTIME for some ideas on tokenization of input, handling null values. Report only your best working prompting strategy and their evaluations. (You are not expected to train/finetune an LLM nor use multimodal LLM i.e., you input PDF or screenshot, your input to the LLM should be PROMPT and a string generated from timeseries dataframe ideally by a script)
- **Part C:** Train a Global Model say prophet or any other. These models are fit once on related time series and used later for prediction, instead of fit being called for every time series separately. You can use all the other columns present in the dataset to create related Time-series.

Again, only report evaluations by this method in the report.

### Part A